In [ ]:
import psycopg2, age, importlib, os
import age.LoadStoreAge.LoadAge as load_age, age.LoadStoreAge.StoreAge as store_age
GRAPH_NAME = 'test_load_store_age'
conn = psycopg2.connect(
    host = 'localhost',
    port = '5432',
    dbname = 'postgres',
    user = 'postgres',
    password = 'postgres'
)
age.setUpAge(conn, GRAPH_NAME)

In [ ]:
importlib.reload(load_age)
importlib.reload(store_age)

In [22]:
def delete_graph_content():
    with conn.cursor() as cursor:
        stmt = '''SELECT * from ag_catalog.cypher('%s', $$ MATCH (u) DETACH DELETE u $$) as (result agtype)''' % (GRAPH_NAME)
        try:
            cursor.execute('''SELECT ag_catalog.drop_graph(%s, true)''', (GRAPH_NAME,))
            cursor.execute('''SELECT ag_catalog.create_graph(%s)''', (GRAPH_NAME,))
            cursor.execute(stmt)
            conn.commit()
        except Exception as ex:
            print(type(ex), ex)

In [23]:
delete_graph_content()

Load the data without nested properties to test loading AGE to CSV and vice versa

In [24]:
label_city, label_country, label_edge = 'City', 'Country', 'EDGE'

stmt1 = '''SELECT create_vlabel('%s', '%s')
                WHERE _label_id('%s', '%s') = 0;''' % (GRAPH_NAME, label_country, GRAPH_NAME, label_country)

stmt2 = '''SELECT create_vlabel('%s', '%s')
                WHERE _label_id('%s', '%s') = 0;''' % (GRAPH_NAME, label_city, GRAPH_NAME, label_city)

stmt3 = '''SELECT create_elabel('%s', '%s')
                WHERE _label_id('%s', '%s') = 0;''' % (GRAPH_NAME, label_edge, GRAPH_NAME, label_edge)

stmt4 = '''SELECT ag_catalog.load_labels_from_file('%s',
                            '%s',
                            'path_to/age/regress/age_load/data/countries.csv');''' % (GRAPH_NAME, label_country)
stmt5 = '''SELECT ag_catalog.load_labels_from_file('%s',
                            '%s',
                            'path_to/age/regress/age_load/data/cities.csv');''' % (GRAPH_NAME, label_city)
stmt6 = '''SELECT ag_catalog.load_edges_from_file('%s',
                            '%s',
                            'path_to/age/regress/age_load/data/edges.csv');''' % (GRAPH_NAME, label_edge)

with conn.cursor() as cursor:
    try:
        cursor.execute(stmt1)
        cursor.execute(stmt2)
        cursor.execute(stmt3)
        cursor.execute(stmt4)
        cursor.execute(stmt5)
        cursor.execute(stmt6)
        conn.commit()
    except Exception as ex:
        print(type(ex), ex)

In [28]:
try:
    os.mkdir('path_to/age/drivers/python/samples/age_load')
except:
    pass
path = 'path_to/age/drivers/python/samples/age_load'

In [29]:
store_age.load_labels_into_file(conn, GRAPH_NAME, path)

In [ ]:
store_age.load_edges_into_file(conn, GRAPH_NAME, path)

In [31]:
delete_graph_content()

In [32]:
load_age.load_labels_from_file(conn, GRAPH_NAME, path + '/data/Country.csv', 'Country')
load_age.load_labels_from_file(conn, GRAPH_NAME,  path + '/data/City.csv', 'City')

In [33]:
load_age.load_edges_from_file(conn, GRAPH_NAME,  path + '/data/EDGE.csv', 'EDGE')

In [34]:
delete_graph_content()

Load the data with nested properties to test loading AGE to JSON and vice versa

In [35]:
total_edges = 50000
with conn.cursor() as cursor:
    for i in range(total_edges):
        try:
            cursor.execute('''
            SELECT * FROM cypher(%s, $$ CREATE (n: NodeA {id : %s,
                key1: {
        subkey1: {
            subsubkey1: %s
        }
    }
    })-[e: EDGE {id: %s,key1: {
        subkey1: {
            subsubkey1: %s
        }
    }}]->(m: NodeB {id : %s, key3: {
        subkey1: {
            subsubkey1: %s
        }}}) $$) AS (result agtype)
            ''', (GRAPH_NAME, str(i), str(i), str(i), str(i), str(i), str(i)))
            conn.commit()
        except Exception as ex:
            print(type(ex), ex)

In [ ]:
store_age.load_labels_into_file(conn, GRAPH_NAME, path)

In [ ]:
store_age.load_edges_into_file(conn, GRAPH_NAME, path)

In [38]:
delete_graph_content()

In [39]:
load_age.load_labels_from_file(conn, GRAPH_NAME,  path + '/data/NodeB.json', 'NodeB')
load_age.load_labels_from_file(conn, GRAPH_NAME,  path + '/data/NodeA.json', 'NodeA')

In [40]:
load_age.load_edges_from_file(conn, GRAPH_NAME,  path + '/data/EDGE.json', 'EDGE')

In [41]:
conn.close()

In [ ]:
import shutil
shutil.rmtree(path)